In [8]:
import pandas as pd
import json

OSky_data = 'OSky_data.csv'
iata_vals = 'icao-iata.csv'

In [9]:
print("proces_started...")
df_whole_flights = pd.read_csv(OSky_data, usecols=['\'icao24\'','\'typecode\'', '\'model\''])
df_icao_iata = pd.read_csv(iata_vals)

df_whole_flights.rename(columns={'\'icao24\'': 'icao', '\'typecode\'': 'typecode', '\'model\'': 'model'}, inplace=True)
df_whole_flights.replace("'", "", regex=True, inplace=True)
df_whole_flights = df_whole_flights[df_whole_flights['typecode'].str.strip() != '']

df_icao_iata.rename(columns={'ICAO\ncode[3]': 'typecode', 'IATA\ntype code': 'iatacode'}, inplace=True)

#make a model database
model_df = df_icao_iata[['Model', 'typecode']]
model_df = model_df.drop_duplicates(subset=['typecode'])
    
df_icao_iata = df_icao_iata.drop(columns = ['Model'])
df_icao_iata = df_icao_iata[(df_icao_iata['iatacode'] != '') & (df_icao_iata['iatacode'] != '—') & (df_icao_iata['iatacode'] != '[to be determined]')]
#pd.set_option('display.max_rows', None)
#display(df_icao_iata)

#merge to get icao-iata correspondance
merged_df = pd.merge(df_whole_flights, df_icao_iata, on='typecode', how='left')
#print(merged_df)

#input the models that are missing from the frist database
for index, row in merged_df.iterrows():
    # Check if the model is empty
    if row['model'] == '':
        # Find the corresponding model from the second dataframe where typecode matches
        corresponding_model = model_df.loc[model_df['typecode'] == row['typecode'], 'Model'].values
        # Update the model in the first dataframe if a corresponding model is found
        if len(corresponding_model) > 0:
            merged_df.at[index, 'model'] = corresponding_model[0]

merged_df.to_csv('combined_databases.csv', index = False)
print(merged_df)
print("proces_finished!")

proces_started...
          icao        model typecode iatacode
0       000001    Antonov 2      AN2      NaN
1       000009    AT-3 R100     AAT3      NaN
2       0000c4  737-448 /SF     B734      734
3       0000c4  737-448 /SF     B734      73P
4       000100      Il-76MD     IL76      IL7
...        ...          ...      ...      ...
519855  faa663                   TWR      NaN
519856  faa66a                   TWR      NaN
519857  faa86b                   TWR      NaN
519858  faafaa                   TWR      NaN
519859  fff20a                   TWR      NaN

[519860 rows x 4 columns]
proces_finished!
